In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import h5py 
import datetime

# Root directory of the project
path = '/mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN'
ROOT_DIR = os.path.abspath(path)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn_chips.config import Config
from mrcnn_chips import utils
import mrcnn_chips.model as modellib
from mrcnn_chips import visualize
from mrcnn_chips.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs_six")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [2]:
print('ROOT_DIR:',ROOT_DIR)
print('MODEL_DIR:',MODEL_DIR)
print('COCO_MODEL_PATH:',COCO_MODEL_PATH)

ROOT_DIR: /mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN
MODEL_DIR: /mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN/logs_six
COCO_MODEL_PATH: /mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN/mask_rcnn_coco.h5


In [3]:
train_path = 'new_train.h5'
test_path = 'new_test.h5'
train_set = h5py.File(train_path,'r')
test_set = h5py.File(test_path,'r')

train_images = train_set['input']
test_images = test_set['input']
train_masks = train_set['output']
test_masks = test_set['output']

del train_set
del test_set
NUM_TRAIN = train_images.shape[0] # 224*224
NUM_TEST = test_images.shape[0]

## Configurations

In [4]:
class ChipsConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "chips"
    
    #CH
    BACKBONE = "resnet50"
    MINI_MASK_SHAPE = (28,28)
    DETECTION_MAX_INSTANCES = 100
    ROI_POSITIVE_RATIO = 0.16
    # USE_MINI_MASK = False # avoid warn of anti-aliasing 
    
    # TRY CH
    POOL_SIZE = 7
    MASK_POOL_SIZE = 14
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1 # CH
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (4, 8, 16, 32, 64)  # anchor side in pixels # CH

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 16 # CH 

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100
        
    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
    
    
config = ChipsConfig()
config.display()

'''
not apply in network yet
ROI_POSITIVE_RATIO = 0.33 # before 0.16
STEPS_PER_EPOCH = 200
'''


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   6]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_class_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES 

'\nnot apply in network yet\nROI_POSITIVE_RATIO = 0.33 # before 0.16\nSTEPS_PER_EPOCH = 200\n'

## Notebook Preferences

In [5]:
def get_ax(rows=1, cols=1, size=8):
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset


In [6]:
class ChipsDataset(utils.Dataset):
    def __init__(self, class_map=None):
        #new_append 
        self.num_null_mask = 0
        self.all_flaw_size = []
        self.subset = ""
        #original
        self._image_ids = []
        self.image_info = []
        # Background is always the first class
        self.class_info = [{"source": "", "id": 0, "name": "BG"}]
        self.source_class_ids = {}

    def load_chips(self, count,subset):
        assert subset in ["train","test"]
        
        self.subset = subset
        
        # Add classes
        self.add_class("chips", 1, "bump")
        self.add_class("chips", 2, "dent")
        self.add_class("chips", 3, "dot")

        # Add images
        # Images are generated on the fly in load_image().
        for i in range(count):
            chips = self.mask_prepare(i)
            self.add_image("chips", image_id=i, path=None,chips=chips)
            
    
    def mask_prepare(self,i,return_mask=False):
        if self.subset == "train":
            mask_full = train_masks[i,:].reshape((4,224,224))
        else :
            mask_full = test_masks[i,:].reshape((4,224,224))
        
        bump,dent,dot = mask_full[1,:,:],mask_full[2,:,:],mask_full[3,:,:]   
        flaw_size = [np.sum(bump),np.sum(dent),np.sum(dot)]
        names = ["bump","dent","dot"]

        if not np.sum(flaw_size):
            self.num_null_mask += 1
            #print("==== no flaw for image %d ===="%i)

        if not return_mask:
            chips= []
            for j in range(3):
                if flaw_size[j] >0:
                    chips.append(names[j])
                    self.all_flaw_size.append(flaw_size[j])
            return chips
        else:
            masks = []
            for j in range(3):
                if flaw_size[j] >0:
                    masks.append(mask_full[j+1,:,:])
            if not np.array(masks).shape[0]:
                return np.array(masks)
            else:
                masks = np.array(masks)
                masks = masks.reshape((-1,224,224))
                masks_reshape = np.zeros((224,224,masks.shape[0]))
                #assert masks.shape[1:] in [(224,224)]
                for i in range(224):
                    for j in range(224):
                        masks_reshape[i,j,:] = masks[:,i,j]  
                return masks_reshape
                      
        
    def load_image(self,image_id):
        info = self.image_info[image_id]
        
        #print("check load_image image_id: ",image_id)
        
        if self.subset == "train":
            image_full = train_images[image_id,:].reshape((6,224,224))
        else:
            image_full = test_images[image_id,:].reshape((6,224,224))
            
        image_full_reshape = np.zeros((224,224,6))
        for i in range(224):
            for j in range(224):
                image_full_reshape[i,j,:] = image_full[:,i,j]  
        image = image_full_reshape
        return image

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        if info["source"] == "chips":
            return info["chips"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        chips = info['chips']
        mask = self.mask_prepare(image_id,True)
        
        class_ids = np.array([self.class_names.index(s) for s in chips])
        return mask.astype(np.bool), class_ids.astype(np.int32)

    def info_display(self,count):
        #flaw_size info
        max_flaw_size = np.max(self.all_flaw_size)
        min_flaw_size = np.min(self.all_flaw_size)
        mean_flaw_size = np.mean(self.all_flaw_size)
        
        #flaw_kind_info
        bump_num,dent_num,dot_num = 0,0,0
        for i in range(count):
            chips = self.image_info[i]['chips']
            if "bump" in chips:
                bump_num +=1
            if "dent" in chips:
                dent_num +=1
            if "dot" in chips:
                dot_num += 1    
                
        #display all info
        print("**** info of %s dataset : ***  "%self.subset)
        print("  ratio of null mask images:%d %%"%(self.num_null_mask/count*100))
        print("  max flaw size:",max_flaw_size)
        print("  min flaw size:",min_flaw_size)
        print("  mean flaw size:",mean_flaw_size)
        print("  bump ratio:%d %%"%(bump_num/count*100))
        print("  dent ratio:%d %%"%(dent_num/count*100))
        print("  dot ratio:%d %%"%(dot_num/count*100))
        

In [7]:
# Testing dataset
dataset_test = ChipsDataset()
dataset_test.load_chips(NUM_TEST,"test")
dataset_test.prepare()
dataset_test.info_display(NUM_TEST)

**** info of test dataset : ***  
  ratio of null mask images:0 %
  max flaw size: 96
  min flaw size: 2
  mean flaw size: 29.385158254918736
  bump ratio:29 %
  dent ratio:41 %
  dot ratio:29 %


In [8]:
#Traning dataset
#cost several mins 
dataset_train = ChipsDataset()
dataset_train.load_chips(NUM_TRAIN,"train")
dataset_train.prepare()
dataset_train.info_display(NUM_TRAIN)

**** info of train dataset : ***  
  ratio of null mask images:0 %
  max flaw size: 102
  min flaw size: 2
  mean flaw size: 29.65715769880092
  bump ratio:28 %
  dent ratio:42 %
  dot ratio:28 %


## Ceate Model

In [12]:
# Create model in training mode
import os
import warnings
warnings.filterwarnings('ignore')
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [13]:
model_path = "/mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN/logs_six/mask_rcnn_chips_0820.h5"
# Load the last model you trained and continue training
model.load_weights(model_path, by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [16]:
start = time.time()
# use_multiprocessing=False
model.train(dataset_train, dataset_test, 
            learning_rate=config.LEARNING_RATE, 
            epochs=300, 
            layers='all')
print("time consuming:",time.time()-start)


Starting at epoch 0. LR=0.001

Checkpoint Path: /mnt/sh_flex_storage/malu/venv/CHIPS_MRCNN/logs_six/chips20180831T1005/mask_rcnn_chips_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b     

Epoch 10/300
100/100 [==============================] - 41s 412ms/step - loss: 1.7226 - rpn_class_loss: 0.0268 - rpn_bbox_loss: 1.2279 - mrcnn_class_loss: 0.0249 - mrcnn_bbox_loss: 0.2231 - mrcnn_mask_loss: 0.2200 - val_loss: 1.6146 - val_rpn_class_loss: 0.0263 - val_rpn_bbox_loss: 1.2511 - val_mrcnn_class_loss: 0.0332 - val_mrcnn_bbox_loss: 0.1669 - val_mrcnn_mask_loss: 0.1372
Epoch 11/300
100/100 [==============================] - 42s 416ms/step - loss: 1.9266 - rpn_class_loss: 0.0259 - rpn_bbox_loss: 1.2678 - mrcnn_class_loss: 0.0251 - mrcnn_bbox_loss: 0.3187 - mrcnn_mask_loss: 0.2891 - val_loss: 1.5682 - val_rpn_class_loss: 0.0231 - val_rpn_bbox_loss: 1.0741 - val_mrcnn_class_loss: 0.0554 - val_mrcnn_bbox_loss: 0.2173 - val_mrcnn_mask_loss: 0.1983
Epoch 12/300
100/100 [==============================] - 42s 420ms/step - loss: 1.8857 - rpn_class_loss: 0.0263 - rpn_bbox_loss: 1.2492 - mrcnn_class_loss: 0.0289 - mrcnn_bbox_loss: 0.2992 - mrcnn_mask_loss: 0.2821 - val_loss: 1.6017 - val

Process Process-76:
Process Process-70:
Process Process-78:
Process Process-72:
Process Process-73:
Process Process-80:
Process Process-83:
Process Process-75:
Process Process-79:
Process Process-71:
Process Process-82:
Process Process-77:
Process Process-85:
Process Process-74:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    s

  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/mnt/sh_flex_storage/malu/venv/lib/python3.5/site-packages/keras/utils/data_utils.py", line 680, in _data_generator_task
    time.sleep(self.wait_time)
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/managers.py", line 717, in _callmethod
    kind, result = conn.recv()
  File "/mnt/sh_flex_storage/malu/venv/lib/python3.5/site-packages/keras/utils/data_utils.py", line 680, in _data_generator_task
    time.sleep(self.wait_time)
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Process Process-8

KeyboardInterrupt: 

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_test, 
            learning_rate=config.LEARNING_RATE,
            epochs=400, 
            layers="heads")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_chips_0820.h5")
model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(ChipsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir= MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
#model_path = os.path.join(MODEL_DIR, "mask_rcnn_chips.h5")
#model_path = model.find_last()

# Load trained weights
model_path = os.path.join(MODEL_DIR, "mask_rcnn_chips_0820.h5")
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_test.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_test, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

original_image_display = original_image[:,:,:3]
visualize.display_instances(original_image_display, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8),colors=[(0.0, 1.0, 1.0)])

In [ ]:
results = model.detect([original_image], verbose=1)
r = results[0]
original_image = original_image[:,:,:3]
visualize.display_instances(original_image_display, r['rois'], r['masks'], r['class_ids'], 
                            dataset_test.class_names, r['scores'], [(0.0, 1.0, 1.0)],ax=get_ax())

In [ ]:
r['masks']

## Evaluation
- After 100 epoch :num_null_mask: 21 mAP:  0.5632911392405063
- After 200 epoch（head）,150 epoch(all):num_null_mask: 6 mAP:  0.9290780141843973

In [ ]:
image_ids = np.random.choice(dataset_test.image_ids, 100)
APs = []
num_null_mask = 0

for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_test, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]

    # Compute AP
    if r['masks'].shape[2]==0:
        num_null_mask += 1
        continue
    else:
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                             r["rois"], r["class_ids"], r["scores"], r['masks'],iou_threshold=0.3)
    APs.append(AP)

print("num_null_mask:",num_null_mask)
print("mAP: ", np.mean(APs))